In [1]:
import pandas as pd
import numpy as np
import pywhatkit as wpp
from datetime import datetime 

#SetOptions para ver los dataframes
#pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth',False)

Lectura del Excel

In [2]:
path = 'Data/'
mes = datetime.now().month

extras = pd.read_excel(path + "Román - M{} Seguimiento.xlsx".format(mes), sheet_name = "Extras")
database = pd.read_excel(path + "BASE CNCI.xlsx", sheet_name = 'Activos')


In [3]:
extras.columns

Index(['Comentario de CE', 'Compromiso ', 'Matricula', 'Nombre',
       'Calificacion', 'Clave', 'Clave Interna', 'Materia', 'Modelo Educativo',
       'Clave Carrera', 'Ciclo', 'Tipo Calificacion', 'Correo', 'Usuario Bb',
       'Estatus Alumno', 'Subproyecto', 'Centro Enlace'],
      dtype='object')

Limpieza de Columnas

In [4]:
extras = extras[["Matricula","Nombre","Calificacion", "Materia","Clave Carrera", "Tipo Calificacion", "Centro Enlace"]]
extras.shape

(28212, 7)

Filtran solo de Cadereyta y CC Cadereyta

In [5]:
extras = extras[extras["Centro Enlace"].isin(["CADEREYTA", "CC CADEREYTA"])]

Se agrupan en una nueva columna las materias del alumno

Se Elimina la fila de Materias al final ya que pasa al frente.
Filtro de duplicados. Renombra la columna Materia por Materia.

In [6]:
df_agrupado = extras.groupby("Nombre")["Materia"].agg(lambda x: ", ".join(x)).reset_index().rename(columns = {"Materia": "Materias"})
extras = pd.merge(extras, df_agrupado, on = "Nombre", how = "left")     

#Cambio de columna y eliminar la extra
columna_extra = extras["Materias"]
extras.drop(columns = ["Materias"], inplace=True)
extras["Materia"] = columna_extra

#Filtra dúplicados
extras = extras.drop_duplicates( subset = "Nombre")
#Renombra
extras.rename(columns = {"Materia" : "Materias"}, inplace= True)

Se eliminan todas las columnas después de la columna CELULAR. 
Se elimina la columna TUTOR ya que está vacía.
Cambia el formato de las columnas TELÉFONO y CELULAR

In [7]:
#Se elimina USUARIO BB,PASSWORD PORTAL ALUMNO, PROYECTO, MODELO EDUCATIVO, SERVICIO, Modalidad, Monto, Correo 2
database.drop(columns = ["USUARIO BB","PASSWORD PORTAL \nALUMNO", 'PROYECTO','MODELO EDUCATIVO', 'Fecha', 'SERVICIO', 'Fecha de inicio','Modalidad', 'Monto', 'Correo 2'], inplace=True)

#Renombrar el nombre de la columna NOMBRE
database.rename(columns = {'NOMBRE                                                                                                                                      ' : "NOMBRE"}, inplace= True)
database.rename(columns = {'MATRICULA' : "Matricula"}, inplace= True)
#Cambia el formato de las columnas TELÉFONO y CELULAR
#df02_final["TELÉFONO"] = df02_final["TELÉFONO"].astype(str).apply(lambda x: "{:.0f}".format(float(x)))
#df02_final["CELULAR"] = df02_final["CELULAR"].astype(str).apply(lambda x: "{:.0f}".format(float(x)))
database

# Filtro Cadereyta
database = database[database['CE CANALIZADO'] == 'CE CADEREYTA']

Lista de los NOMBRES y MATRICULAS de los Reprobados

In [8]:
matricula_reprobados = extras['Matricula'].unique()
matricula_reprobados

array(['UVNL019745', 'UVNL022209', 'UVNL022852', 'UVNL029717', 'AL041577',
       'AL056425', 'AL067659', 'AL073152', 'AL082106', 'AL084790',
       'AL085284', 'AL088323', 'AL089408', 'AL089432', 'AL093169',
       'AL094015', 'BNL044964', 'BNL049990', 'BNL050006', 'BNL052841',
       'BNL053601', 'BNL055373', 'BNL055698', 'BNL062194', 'BNL064346',
       'BNL065238', 'BNL067709', 'BNL068604', 'BNL068661', 'BNL069453',
       'BNL070153', 'BNL071412', 'BNL072228', 'BNL072590', 'BNL073225',
       'BNL073643', 'BNL073667', 'BNL075497', 'BNL076176', 'BNL077061',
       'BNL078039', 'BNL079305', 'BNL080024', 'BNL081348', 'BNL081349',
       'BNL081995', 'BNL083647', 'BNL085115', 'BNL086218', 'BNL087363',
       'BNL087421', 'BNL087428', 'BNL087758', 'BNL088083', 'BNL089250',
       'BNL090068', 'BNL091793', 'BNL091796', 'BNL092420', 'BNL092815',
       'BNL093152', 'BNL093782', 'BNL093966', 'BNL094177', 'BNL094370',
       'BNL095474', 'BNL095997', 'BNL096292', 'BNL096876', 'BNL098173',


Reprobados que están en el Módulo 4

In [9]:
extras.columns

Index(['Matricula', 'Nombre', 'Calificacion', 'Materias', 'Clave Carrera',
       'Tipo Calificacion', 'Centro Enlace'],
      dtype='object')

Tabla comprendida en la cual solo está la información de los reprobados que están en Módulo 4

In [10]:
reprobados_mod4 = database[(database['Matricula'].isin(matricula_reprobados)) & (database['M4'] == 4)]
reprobados_mod4 = pd.merge(extras, reprobados_mod4, on = 'Matricula')
reprobados_mod4.columns

Index(['Matricula', 'Nombre', 'Calificacion', 'Materias', 'Clave Carrera',
       'Tipo Calificacion', 'Centro Enlace', 'NOMBRE', 'CE CANALIZADO',
       'CARRERA', 'M4', 'Correo', 'Teléfono', 'Teléfono2', 'Teléfono3', 'CURP',
       'Agente ', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar'],
      dtype='object')

In [11]:
reprobados_mod4 = reprobados_mod4[['Matricula', 'Nombre', 'Calificacion', 'Materias', 'Clave Carrera', 'Tipo Calificacion', 'Centro Enlace', 'M4', 'Correo', 'Teléfono', 'Teléfono2', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

In [12]:
reprobados_mod4.shape

(21, 14)

#### Archivos de utilidad

In [13]:
path2 = 'infodata/'
try: 
    matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
    matric_yacontactadas = pd.read_csv(path2 + 'matriculas_yacontactadas.csv')
    
    matric_error = matric_error.dropna().values.flatten().tolist()
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist()
except:
    matric_error = pd.DataFrame
    matric_yacontactadas = pd.DataFrame
    
print(matric_error)

['BNL099575', 'BNL101971', 'BNL103357', 'BNL103889', 'AL103269', 'AL103798', 'BNL100961', 'BNL102647', 'BNL103330', 'BNL103890', 'BNL065238', 'BNL032654', 'AL069900', 'AL089432', 'AL093752', 'AL094015', 'BNL085130', 'BNL088729', 'BNL095698', 'AL085284', 'BNL083647']


Mensaje predeterminado para alumnos

In [14]:
for index, row in reprobados_mod4.iterrows():
    alumno = row['Nombre']
    matricula = row['Matricula']
    tipoExamen = row['Tipo Calificacion']
    materias = row['Materias']
    telefono = row['Teléfono']
    
    if not np.isnan(telefono):
        telefono = '+52 '+str(telefono)
    if matricula in (matric_error):
        if not np.isnan(row['Teléfono2']):
            telefono2 ='+52 '+str(row['Teléfono2'])
            # print(telefono, telefono2)
            telefono = telefono2

    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, alumno: {}'.format(matricula, alumno))
    else:
        mensaje = 'Se acerca la fecha de Extraordinarios. ¿Cómo te encuentras ' + alumno + ' ' + matricula + '? \nTe habla Román Rodríguez Tutor Presencial del CNCI, me comunico contigo debido a que veo que dejaste en ' + tipoExamen + ' la(s) siguiente(s) materia(s): ' + materias +'. \n¿Me gustaría saber si puedo apoyarte con el proceso de regularización? \nTe dejo las fechas siguientes:'
        # mensaje2 = 'Hola, que tal ' + alumno + ' ' + matricula_reprobados + ', soy Román Rodríguez tu Tutor Presencial del CNCI, me comunico contigo debido a que veo que dejaste en ' + tipoExamen + 'la(s) siguiente(s) materia(s): ' + materias +'. ¿Me gustaría saber si puedo apoyarte con el proceso de tus materias en extraordinario?.'
        print('Enviado a: {}, {}'.format(matricula, alumno))
        # wpp.sendwhat_image(telefono, imagen, mensaje, 25, True)

Matricula antes contactada: UVNL029717, alumno: CARLOS VELAZQUEZ BANDA
Matricula antes contactada: AL067659, alumno: MARVID HANIEL ACOSTA RODRIGUEZ
Enviado a: AL073152, CHRISTOFER ROBIN SOTO SANCHEZ
Matricula antes contactada: AL084790, alumno: KEVIN ALEJANDRO VEGA VEGA
Matricula antes contactada: AL085284, alumno: LAURA EDITH MORENO GAMEZ
Matricula antes contactada: AL089408, alumno: ALESSANDRA ELIZABETH SALAZAR HERNANDEZ
Matricula antes contactada: BNL064346, alumno: DIEGO ALBERTO GALVAN GONZALEZ
Matricula antes contactada: BNL067709, alumno: MAYRA ADRIANA  BETANCOURT HERNANDEZ
Matricula antes contactada: BNL071412, alumno: PAOLA MARGARITA LOREDO GONZALEZ
Enviado a: BNL072590, RAMIRO RAMOS SANTOS
Matricula antes contactada: BNL073667, alumno: JORGE EDUARDO ALVARADO CAVAZOS
Matricula antes contactada: BNL075497, alumno: ANGELA JAZMIN CARRILLO VAZQUEZ
Matricula antes contactada: BNL083647, alumno: FRANCISCO VILLA RODRIGUEZ
Matricula antes contactada: BNL085115, alumno: SCARLETT AMAYRAN

# Consultas

In [15]:
extras[extras['Matricula']== 'BNL088083']

,Matricula,Nombre,Calificacion,Materias,Clave Carrera,Tipo Calificacion,Centro Enlace
236,BNL088083,CARLOS ALBERTO DIAZ DAMASO,6,MÉTODOS NUMÉRICOS,IGE,Extraordinario,CC CADEREYTA


In [16]:
database[database['Matricula']== 'BNL088083']

,NOMBRE,Matricula,CE CANALIZADO,CARRERA,M4,Correo,Teléfono,Teléfono2,Teléfono3,CURP,Agente,Asesor Académico,Asesor Financiero,Asesor Escolar
613,CARLOS ALBERTO DIAZ DAMASO,BNL088083,CE CADEREYTA,IGE,1,cadd_43@hotmail.com,8187772623,NaN,NaN,DIDC950206HVZZMR07,Coordinador Cadereyta,Cecilia Rodriguez,Saúl Garza,Angel Sandoval
